CRISPR TXTL Experiment: comparing 1 or more different RNA guides across a range of concentrations


In [1]:
experiment_name = 'Linda-CPF1-ds_vs_ss'

# set your reaction volume:

reaction_volume = 5e-6 # 2 microliters

# set your mastermix ratio

master_mix_ratio = 0.60 # 75% 

# enter concentrations for your starting source solutions

# enter your desired finnal concentration(s)
chi_concentration = 2.5e-6
nuclease_dna_concentration = 1e-9

experiment_polymerase_dna_concentration = [0.1e-9, 0.5e-9]
experiment_reporter_dna_concentrations = [2.0e-9]
experiment_guideRNA_dna_concentration = [1e-9]

# create labels for each of your guides to test
targets = [
    'dsDNA1',
    'dsDNA2',
    'dsDNA7',
    'dsDNANT',
    'ssDNA1',
    'ssDNA2',
    'ssDNA7',
    'ssDNANT',  
]

# fixed concentrations
experiment_starting_conc = {
    'chi' : 30e-6,
    'reporter' : 12e-9,
    'nuclease' : 100e-9,
    'polymerase' : 9.8e-9,
    'no guide control' : 0,
    'no reporter control' : 0,
    'blank control' : 0,
}

experiment_rna_guides = targets

# and finally set the number of replicates for each run

replicates = 2

# plate layout
dest_layout = (12,8) # 96 well
# dest_layout = (24,16) # 384 

src_layout = (24,16) # 384

# we assume you're using the standard source plate (15-65µL working range)
b_use_ldv = False
if b_use_ldv:
    transfer_range = (2.5e-6, 12e-6)
else:
    transfer_range = (15e-6, 65e-6)

In [2]:
# import python things here
import pandas as pd
from IPython.core.display import display, HTML
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [425]:
class EchoManager():
    
    def __init__(self, src_layout, src_transfer_range, dest_layout, reaction_volume, 
                 master_mix_ratio=0.75, 
                 starting_concentrations={}):
        
        
        ##TODO refactor
        self.starting_concentration = starting_concentrations
        
        self.src_aliquots = {}
        self.aliquots = []
        self.plate_row_index = 0
        self.dst_plate_index = 0
        
        self.min_transfer_size = 2.5 * 1e-3 # captured, but not needed 
        self.reaction_volume = reaction_volume
        self.txtl_mix_volume = reaction_volume * master_mix_ratio
        self.available_input_volume = reaction_volume - self.txtl_mix_volume
        
        self.src_layout = src_layout
        self.dest_layout = dest_layout
        self.src_transfer_range = src_transfer_range
        
        self.final_source_df = None
        self.effective_transfer_volume = src_transfer_range[1] - src_transfer_range[0]
            
    def src_plating_index_to_str(self, index):
        row = floor(index/self.src_layout[0])
        col = index % self.src_layout[0]
        return '%s%02d' % (chr(int(row + 65)), col+1)

    def dst_plating_index_to_str(self, index):
        row = floor(index/self.dest_layout[0])
        col = index % self.dest_layout[0]
        return '%s%02d' % (chr(int(row + 65)), col+1)
    

    def add_rna_targets(self, targets):
        for t in targets:
            self.starting_concentration[t] = 24e-9

    def generate_source_plating(self):
        final_source_plating = []

        for k,v in self.src_aliquots.items():
            for a in v:        
                user_step = {
                    'reagent' : k,
                    'well' : a['well_str'],
                    'volume' : (a['active_volume_ul']),
                    'active_volume_ul' : (a['active_volume_ul']) / 1e-6,
                    'volume_plus' : a['active_volume_ul'] + self.src_transfer_range[0]            
                }

                user_step['volume_ul']  = user_step['volume_plus'] / 1e-6
                final_source_plating.append(user_step)
                
        self.final_source_df = pd.DataFrame(final_source_plating)
        
        return self.final_source_df.sort_values(by='well')
    
    def generate_transfers(self):
        
        transfer_df = pd.DataFrame(self.aliquots)
        self.transfer_df = transfer_df
        
        final_xfers = []
        for n,transfer in transfer_df.iterrows():

            for rgt in ['master', 'chi', 'h20', 'polymerase', 'nuclease', 'rna', 'reporter']:

                rgt_source_well = '%s_source' % rgt
                rgt_source_name = '%s_source_name' % rgt
                rgt_source_vol = '%s_ul' % rgt

                xfer = {
                    'condition' : transfer['name'],

                    'rgt' : rgt,
                    'source_name' : transfer[rgt_source_name],
                    'dst_volume' : transfer[rgt_source_vol] / 1e-9, # we output the csv in nL
                    'dst_plate_label' : 'dstPlate1',
                    'src_plate_label' : 'srcPlate1',
                    'src_well' : transfer[rgt_source_well],
                    'dst_well' : transfer['index_str'], 
                    'pk' : transfer['index']
                }
                
                if transfer[rgt_source_vol] > 0:
                    final_xfers.append(xfer) 
                
    
        RGT_RANK = {
            'master' : 0,
            'chi' : 1,
            'h20' : 2,
            'polymerase' : 3,
            'reporter' : 4,         
            'nuclease' : 5,
            'rna' : 6
        }    
        
        def rank_by_reagent(row):
            if row['rgt'] in RGT_RANK:
                return RGT_RANK[row['rgt']]
            return 999

        def plate_transfer_by_reagent(row):
            if row['rgt'] == 'master':
                return '384PP_AQ_CP'
            return '384PP_AQ_BP2'
        
        final_xfers_df = pd.DataFrame(final_xfers)
        final_xfers_df['rgt_rank'] = final_xfers_df.apply(rank_by_reagent, axis=1)
        final_xfers_df['Source Plate Type'] = final_xfers_df.apply(plate_transfer_by_reagent, axis=1)
        
        final_xfers_df = final_xfers_df.sort_values(by=['rgt_rank', 'pk'])        
        self.final_xfers_df = final_xfers_df
        
    def generate_pre_chi_transfer(self):
        prechi = self.final_xfers_df[self.final_xfers_df.rgt_rank < 4][['condition','src_plate_label', 'src_well', 'dst_volume', 'dst_plate_label', 'dst_well', 'Source Plate Type']]
        prechi.columns = ['Sample ID', 'Source Plate Name', 'Source Well', 'Transfer Volume', 'Destination Plate Name', 'Destination Well', 'Source Plate Type']        
        return prechi

    def generate_post_chi_transfer(self):
        postchi = self.final_xfers_df[self.final_xfers_df.rgt_rank >= 4][['condition', 'src_plate_label', 'src_well', 'dst_volume', 'dst_plate_label', 'dst_well', 'Source Plate Type']]
        postchi.columns = ['Sample ID', 'Source Plate Name', 'Source Well', 'Transfer Volume', 'Destination Plate Name', 'Destination Well', 'Source Plate Type']
        return postchi
        
    def create_aliquots_for_crispr_txtl(self, rna_guides, reporter_dna_concentrations, guideRNA_dna_concentration, polymerase_dna_concentration):
                    
            for guide_name in rna_guides:
                for dna_conc in reporter_dna_concentrations:
                    for rna_conc in guideRNA_dna_concentration:
                        
                        if 'ssDNA' in guide_name:
                            poly_concentrations = polymerase_dna_concentration
                        else: 
                            poly_concentrations = [0]
                        
#                         print(guide_name, dna_conc, rna_conc, poly_concentrations)
                        
                        for n,poly_conc in enumerate(poly_concentrations):
                            
                            nmol_reporter = dna_conc * reaction_volume
                            nmol_rna = rna_conc * reaction_volume
                            nmol_chi = chi_concentration * reaction_volume
                            nmol_nuclease = nuclease_dna_concentration * reaction_volume
                            
                            if 'ss' in guide_name:
                                nmol_polymerase = poly_conc * reaction_volume
                                actual_polycom = poly_conc
                            else:
                                nmol_polymerase = 0
                                actual_polycom = 0

                            if self.starting_concentration['reporter'] > 0:
                                ul_reporter = nmol_reporter / self.starting_concentration['reporter']
                            else:
                                ul_reporter = 0

                            if self.starting_concentration[guide_name] > 0:
                                ul_rna = nmol_rna / self.starting_concentration[guide_name]
                            else:
                                ul_rna = 0

                            ul_nuclease = nmol_nuclease / self.starting_concentration['nuclease']
                            ul_polymerase = nmol_polymerase / self.starting_concentration['polymerase']
                            ul_chi = nmol_chi / self.starting_concentration['chi'] 

                            ul_final_h20 = self.reaction_volume - self.txtl_mix_volume - (ul_reporter + ul_rna + ul_chi + ul_nuclease + ul_polymerase)        

                            for rep in range(replicates):

                                aliquot_name = '%s - %0.2fnM-rep x %0.2fnM guide x %0.2f nM pol (%d)' % (guide_name, dna_conc*1e9, rna_conc*1e9, actual_polycom*1e9, rep)

                                aliquot = {
                                    'guide': guide_name,
                                    'name' : aliquot_name,
                                    'reporter_ul' : ul_reporter,
                                    'master_ul' : self.txtl_mix_volume,
                                    'nuclease_ul': ul_nuclease,
                                    'polymerase_ul' : ul_polymerase,
                                    'rna_ul' : ul_rna,
                                    'chi_ul' : ul_chi,
                                    'h20_ul' : ul_final_h20,
                                    'index' : self.dst_plate_index,
                                    'index_str' : self.dst_plating_index_to_str(self.dst_plate_index)
                                }


                                # increment the plate index to the next loction
                                self.dst_plate_index += 1

                                for rgt in ['master', 'chi', 'h20', 'polymerase', 'nuclease', 'rna', 'reporter']:

                                    # what is the reagent key to use?
                                    if 'rna' in rgt:
                                        aliquot_rgt_name = '%s rna' % (guide_name)
                                    else:
                                        aliquot_rgt_name = rgt

                                    # how much volume is needed for this reagent?
                                    transfer_needed = aliquot[rgt + '_ul']

                                    # if the transfer is > 0uL
#                                     if transfer_needed > 0:

                                    # step 1: if no aliquots are present for this reagent, create a list of them
                                    if aliquot_rgt_name not in self.src_aliquots:
                                        self.src_aliquots[aliquot_rgt_name] = []


                                    # step 2: if that list is empty, initialize the aliquot 
                                    if len(self.src_aliquots[aliquot_rgt_name]) == 0 :

                                        new_src_aliquot = {
                                            'reagent' : aliquot_rgt_name,
                                            'well_str' : self.src_plating_index_to_str(self.plate_row_index),
                                            'aliquot_index' : -1, 
                                            'active_volume_ul' : 0,
                                            'physical_volume_ul' : 0
                                        }                    

                                        self.plate_row_index += 1 
                                        self.src_aliquots[aliquot_rgt_name].append(new_src_aliquot)                                                    

                                    # step 3: check if the transfer requires more volume than is accessible in this aliquot

                                    current_aliquot_ptr = self.src_aliquots[aliquot_rgt_name][-1] # get the last aliquot in the list                         
                                    aliquot_index = self.src_aliquots[aliquot_rgt_name].index(current_aliquot_ptr)

                                    if (transfer_needed + current_aliquot_ptr['active_volume_ul']) > self.effective_transfer_volume:

                                        # no, we don't have enough space left, need to create a new aliquot

                                        # if the transfer would overfill the source, create a new one and put it there instead
                                        new_src_aliquot = {
                                            'reagent' : aliquot_rgt_name,
                                            'aliquot_index' : aliquot_index,
                                            'well_str' : self.src_plating_index_to_str(self.plate_row_index),
                                            'active_volume_ul' : transfer_needed,
                                            'physical_volume_ul' : 0
                                        }  

                                        self.plate_row_index += 1                                    
                                        self.src_aliquots[aliquot_rgt_name].append(new_src_aliquot)

                                        current_aliquot_ptr = self.src_aliquots[aliquot_rgt_name][-1] # get the last aliquot in the list                         
                                        aliquot_index = self.src_aliquots[aliquot_rgt_name].index(current_aliquot_ptr)

                                        aliquot[rgt + '_source'] = current_aliquot_ptr['well_str']
                                        aliquot[rgt + '_source_name'] = '%s %d %s' % (aliquot_rgt_name, current_aliquot_ptr['aliquot_index'], current_aliquot_ptr['well_str'])

                                    else:

                                        # yes we have enough, just add 
                                        current_aliquot_ptr['active_volume_ul'] += transfer_needed
                                        current_aliquot_ptr['aliquot_index'] = aliquot_index

                                        aliquot[rgt + '_source'] = current_aliquot_ptr['well_str']
                                        aliquot[rgt + '_source_name'] = '%s %d %s' % (aliquot_rgt_name, current_aliquot_ptr['aliquot_index'], current_aliquot_ptr['well_str'])

#                                         print("Using existing  aliquot", current_aliquot_ptr)
#                                         print(aliquot)
#                                         print('------------------------------------------------------\n')

                                self.aliquots.append(aliquot)
    
    

In [426]:

# we instantiate the EchoMananger with some parameters

echo = EchoManager(src_layout=src_layout, 
                src_transfer_range=transfer_range,
                dest_layout=dest_layout,
                reaction_volume=reaction_volume,
                master_mix_ratio=master_mix_ratio,
                starting_concentrations=experiment_starting_conc
               )

# add our targets. Our guide source DNA is 24nM
echo.add_rna_targets(experiment_rna_guides)

# create the experimental aliquots
echo.create_aliquots_for_crispr_txtl(experiment_rna_guides, experiment_reporter_dna_concentrations, experiment_guideRNA_dna_concentration, experiment_polymerase_dna_concentration)

# create controls with no guide source DNA
echo.create_aliquots_for_crispr_txtl(['no guide control'], experiment_reporter_dna_concentrations, [0], [0])

# create controls with no reporter DNA present
echo.create_aliquots_for_crispr_txtl(['blank control'], [0], [0], [0])

# print out a quick list of the required volumes for our source plate
echo.generate_source_plating().sort_values(by='well')

,active_volume_ul,reagent,volume,volume_plus,volume_ul,well
11,48.000000,master,4.800000e-05,0.000063,63.000000,A01
5,11.666667,chi,1.166667e-05,0.000027,26.666667,A02
8,13.817687,h20,1.381769e-05,0.000029,28.817687,A03
9,2.448980,polymerase,2.448980e-06,0.000017,17.448980,A04
0,1.400000,nuclease,1.400000e-06,0.000016,16.400000,A05
7,0.416667,dsDNA1 rna,4.166667e-07,0.000015,15.416667,A06
4,21.666667,reporter,2.166667e-05,0.000037,36.666667,A07
14,0.416667,dsDNA2 rna,4.166667e-07,0.000015,15.416667,A08
6,0.416667,dsDNA7 rna,4.166667e-07,0.000015,15.416667,A09
13,0.416667,dsDNANT rna,4.166667e-07,0.000015,15.416667,A10


In [443]:
echo.generate_transfers() # this takes the available sources and requirements and generates the transfer lists for pre & post Chi incubatio

In [446]:
# here is a list transfers needed for each experimental sample.
echo.transfer_df.head()

,chi_source,chi_source_name,chi_ul,guide,h20_source,h20_source_name,h20_ul,index,index_str,master_source,...,nuclease_ul,polymerase_source,polymerase_source_name,polymerase_ul,reporter_source,reporter_source_name,reporter_ul,rna_source,rna_source_name,rna_ul
0,A02,chi 0 A02,4.166667e-07,dsDNA1,A03,h20 0 A03,4.916667e-07,0,A01,A01,...,5.000000e-08,A04,polymerase 0 A04,0.0,A07,reporter 0 A07,8.333333e-07,A06,dsDNA1 rna 0 A06,2.083333e-07
1,A02,chi 0 A02,4.166667e-07,dsDNA1,A03,h20 0 A03,4.916667e-07,1,A02,A01,...,5.000000e-08,A04,polymerase 0 A04,0.0,A07,reporter 0 A07,8.333333e-07,A06,dsDNA1 rna 0 A06,2.083333e-07
2,A02,chi 0 A02,4.166667e-07,dsDNA2,A03,h20 0 A03,4.916667e-07,2,A03,A01,...,5.000000e-08,A04,polymerase 0 A04,0.0,A07,reporter 0 A07,8.333333e-07,A08,dsDNA2 rna 0 A08,2.083333e-07
3,A02,chi 0 A02,4.166667e-07,dsDNA2,A03,h20 0 A03,4.916667e-07,3,A04,A01,...,5.000000e-08,A04,polymerase 0 A04,0.0,A07,reporter 0 A07,8.333333e-07,A08,dsDNA2 rna 0 A08,2.083333e-07
4,A02,chi 0 A02,4.166667e-07,dsDNA7,A03,h20 0 A03,4.916667e-07,4,A05,A01,...,5.000000e-08,A04,polymerase 0 A04,0.0,A07,reporter 0 A07,8.333333e-07,A09,dsDNA7 rna 0 A09,2.083333e-07


In [431]:
source_plates = echo.final_source_df
prechi = echo.generate_pre_chi_transfer()
postchi = echo.generate_post_chi_transfer()
final_samples = pd.DataFrame(echo.aliquots)

source_plating_path = './echo_planning/%s-source-plating.csv' % experiment_name
first_transfer_path = './echo_planning/%s-prechi.csv' % experiment_name
second_transfer_path = './echo_planning/%s-postchi.csv' % experiment_name
final_samples_path = './echo_planning/%s-sample-list.csv' % experiment_name

source_plates.to_csv(source_plating_path, index=False)
prechi.to_csv(first_transfer_path, index=False)
postchi.to_csv(second_transfer_path, index=False)
final_samples.to_csv(final_samples_path, index=False)

<h1><a style="font-size: 150%;" target="_blank" href="echo_planning">Click here to download the generated CSVs</a></h1>

They will be named with the prefix of the experiment (defined at the very top)